In [52]:
import pandas as pd
from py4j.java_gateway import JavaGateway
import psycopg2
import numpy as np

In [53]:
home = "/Users/liviaalmada/NYDataset/Link Level Traffic Estimates/"
#home = "/media/livia/DATA/Dropbox/TRABALHO/PESQUISA/Doutorado/Dados/NY-Data/"

In [54]:
graphnodes = pd.read_csv(home+"nodes.csv")

In [55]:
graphlinks = pd.read_csv(home+"/links.csv", delimiter=',')

In [56]:
graphnodes.head()

,node_id,is_complete,num_in_links,num_out_links,osm_traffic_controller,xcoord,ycoord,osm_changeset,birth_timestamp,death_timestamp,grid_region_id
0,42467266,t,3,3,null,-73.917419,40.660051,3218174,1405267167641,40000000000000,225
1,42467262,t,4,4,null,-73.918294,40.659484,3218174,1405267167641,40000000000000,225
2,42467259,t,3,3,null,-73.919089,40.658966,3218174,1405267167641,40000000000000,205
3,42467258,t,3,3,null,-73.919968,40.658398,3218174,1405267167641,40000000000000,205
4,42467250,t,2,2,null,-74.005260,40.600091,7948780,1405267167641,40000000000000,143


In [57]:
graphlinks.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


### Create Graph

In [58]:
gateway = JavaGateway() 

In [59]:
gateway.newGraph("ny", home+"graph")

In [60]:
graph = gateway.getGraph("ny");

In [61]:
graph.setNumberOfIntervals(24)

In [62]:
nodesMap = { }


In [63]:
idn = 0
for n in graphnodes.values:
    #gateway.addNode(n[0], n[5], n[6])
    nodesMap[n[0]] = idn
    idn = idn + 1

In [64]:
graph.getNumberOfNodes()

0

In [65]:
print(nodesMap.get(0))

None


In [66]:
nodesMap = dict()
idn = 0
for (idx, row) in graphlinks.iterrows():
    
    if(nodesMap.get(row['begin_node_id']) is None):
        gateway.addNode( row['begin_node_id'] , row['startY'] , row['startX'])
        nodesMap[row['begin_node_id']] = idn
        idn = idn + 1
    if(nodesMap.get(row['end_node_id']) is None):
        gateway.addNode( row['end_node_id'] , row['endY'] , row['endX'])
        nodesMap[row['end_node_id']] = idn
        idn = idn + 1
    gateway.addEdge(nodesMap.get(row['begin_node_id']), nodesMap.get(row['end_node_id']), row['street_length'])
       

In [16]:
graph.getNumberOfEdges()

260855

In [17]:
graph.getNumberOfNodes()

96570

In [18]:
graphcosts = pd.read_csv(home+"/links.csv", delimiter=',')

In [19]:
graphcosts.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


In [20]:
grouped = graphcosts.groupby('begin_node_id')

In [21]:
grouped.ngroups

96435

## Generate graph costs

In [67]:
conn = psycopg2.connect(host="localhost", database="newyork", user="postgres")

In [68]:
curs = conn.cursor()

In [69]:
curs.execute("select begin_node_id, end_node_id, meantt, stdtt, hourday from gaussian_tt where weekday = 0 and begin_node_id<>0")

In [70]:
costs = curs.fetchall()

In [71]:
costs
curs.close()

In [72]:
costs

[(42421728, 42421731, 29.2340708333333, 12.2130202617766, 0.0),
 (42421728, 42421731, 29.2190733333333, 15.5894640257437, 1.0),
 (42421728, 42421731, 24.9455734782609, 13.6831009891197, 2.0),
 (42421728, 42421731, 26.8643666666667, 15.362794654735, 3.0),
 (42421728, 42421731, 21.2376775, 6.66165366131742, 4.0),
 (42421728, 42421731, 25.6167964, 19.7371788147889, 5.0),
 (42421728, 42421731, 17.8204385, 9.82056517095948, 6.0),
 (42421728, 42421731, 20.8674557894737, 9.94771926074849, 7.0),
 (42421728, 42421731, 23.6180142857143, 13.4206258308863, 8.0),
 (42421728, 42421731, 30.2520967741935, 13.9256408857785, 9.0),
 (42421728, 42421731, 31.5452595454545, 14.9290839226295, 10.0),
 (42421728, 42421731, 28.969168, 11.2229418623717, 11.0),
 (42421728, 42421731, 30.2013826086957, 11.6543178756785, 12.0),
 (42421728, 42421731, 35.78686625, 15.8734310550717, 13.0),
 (42421728, 42421731, 26.4413173913043, 8.45118522635332, 14.0),
 (42421728, 42421731, 33.1927222222222, 12.209105245383, 15.0),
 (

In [73]:
for cost in costs:
    node_from = nodesMap.get(cost[0])
    node_to = nodesMap.get(cost[1])
    meantt = cost[2]
    stdtt = cost[3]
    hourday = cost[4]
    gateway.addGaussianCost(node_from,node_to,meantt, stdtt, int(hourday))

In [74]:
x = graph.getProbabilisticCosts(1)

In [75]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link-5-10-15-10", 0.5, 10, 15, 10)

In [76]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link-4-10-15-10", 0.4, 10, 15, 10)

In [77]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link-3-10-15-10", 0.3, 10, 15, 10)

In [78]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link-2-10-15-10", 0.2, 10, 15, 10)

In [79]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link-1-10-15-10", 0.1, 10, 15, 10)

In [80]:
gateway.runLinkScanCorrelation("ny", home+"linkcos-1-10-15-10", 0.1, 10, 15, 10)
gateway.runLinkScanCorrelation("ny", home+"linkcos-2-10-15-10", 0.2, 10, 15, 10)
gateway.runLinkScanCorrelation("ny", home+"linkcos-3-10-15-10", 0.3, 10, 15, 10)
gateway.runLinkScanCorrelation("ny", home+"linkcos-4-10-15-10", 0.4, 10, 15, 10)
gateway.runLinkScanCorrelation("ny", home+"linkcos-5-10-15-10", 0.5, 10, 15, 10)